In [ ]:
import pandas as pd
import json
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # type: ignore
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import tqdm as notebook_tqdm

In [ ]:
# Load the chat history JSON file
with open('./data/chat_history.json', 'r') as file:
    chat_data = json.load(file)

In [ ]:
chat_data

In [ ]:
# Clean the text data
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.lower()  # Convert to lowercase
    return text

In [ ]:
# Sentiment analysis
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

In [ ]:
# Adding cleaned text to chat_data 
for customer in chat_data:
    for message in customer['chat_history']:
        if 'message' in message:
            message['cleaned_text'] = clean_text(message['message'])
        elif 'agent_message' in message:
            message['cleaned_text'] = clean_text(message['agent_message'])

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)



# Playground 

In [ ]:
# Topic modeling
all_texts = [message['cleaned_text'] for customer in chat_data for message in customer['chat_history'] if 'cleaned_text' in message]


In [ ]:
all_texts

In [ ]:

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(all_texts)


In [ ]:
X

In [ ]:
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

In [ ]:
def print_topics(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))



In [ ]:
lda.components_[3]

In [ ]:
print_topics(lda, vectorizer.get_feature_names_out(), 3)

In [ ]:
message['cleaned_text']

In [ ]:
customer

In [ ]:
chat_data